In [ ]:
# import libraries

import json
import os
import cv2
import math
import matplotlib.pyplot as plt
import mtcnn

In [ ]:
directory = r'D:\Professional\Computer Engineering\Final Year Project\Dataset_Iteration1'
frameNr = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    cap = cv2.VideoCapture(f)
    count = 0    
    frame_rate = cap.get(5) #frame rate
    while(cap.isOpened()):
        #print(frameNr)
        #print("Inside loop")
        frame_id = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break

        if (frame_id % math.floor(frame_rate) == 0):
            print('Original Dimensions: ', frame.shape)
            if frame.shape[1] < 300:
                scale_ratio = 2
            elif frame.shape[1] > 1900:
                scale_ratio = 0.33
            elif frame.shape[1] > 1000 and frame.shape[1] <= 1900 :
                scale_ratio = 0.5
            else:
                scale_ratio = 1
            print('Scale Ratio: ', scale_ratio)

            width = int(frame.shape[1] * scale_ratio)
            height = int(frame.shape[0] * scale_ratio)
            dim = (width, height)
            new_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            print('Resized Dimensions: ', new_frame.shape)
            #cv2.imshow("Frame",new_frame)
            #new_filename = '{}-{:03d}.png'.format(os.path.join(tmp_path, get_filename_only(filename)), count)
            count = count + 1 
            path = r"D:\Professional\Computer Engineering\Final Year Project\Frames_Iteration1\frame"+str(frameNr)+".jpg"
            #print(path)
            cv2.imwrite(path,new_frame)
            frameNr += 1
    cap.release()

In [ ]:
# draw an image with detected objects
def draw_facebox(filename, result_list):
    # load the image
    data = plt.imread(filename)
    # plot the image
    plt.imshow(data)
    # get the context for drawing boxes
    ax = plt.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = plt.Rectangle((x, y), width, height, fill=False, color='green')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    plt.show() 

directory = r'D:\Professional\Computer Engineering\Final Year Project\Frames_Iteration1'
count = 0
detector = mtcnn.MTCNN()
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    pixels = plt.imread(f)
    print("Shape of image/array:",pixels.shape)
    imgplot = plt.imshow(pixels)
    plt.show()
    
    # detect faces in the image
    faces = detector.detect_faces(pixels)
    print('Face Detected: ', len(faces))
    print(faces)
    # display faces on the original image
    draw_facebox(f, faces)
    name_count = 1
    for face in faces:
            bounding_box = face['box']
            print(bounding_box)
            confidence = face['confidence']
            print(confidence)
            if len(faces) < 2 or confidence > 0.95:
                margin_x = bounding_box[2] * 0.3  # 30% as the margin
                margin_y = bounding_box[3] * 0.3  # 30% as the margin
                x1 = int(bounding_box[0] - margin_x)
                if x1 < 0:
                    x1 = 0
                x2 = int(bounding_box[0] + bounding_box[2] + margin_x)
                if x2 > pixels.shape[1]:
                    x2 = pixels.shape[1]
                y1 = int(bounding_box[1] - margin_y)
                if y1 < 0:
                    y1 = 0
                y2 = int(bounding_box[1] + bounding_box[3] + margin_y)
                if y2 > pixels.shape[0]:
                    y2 = pixels.shape[0]
                print(x1, y1, x2, y2)
                crop_image = pixels[y1:y2, x1:x2]
                #temp = cv2.cvtColor(crop_image, cv2.COLOR_RGB2BGR)
                cv2.imshow("Cropped Image",crop_image)
                cv2.waitKey(5)
                dir = r'D:\Professional\Computer Engineering\Final Year Project\Faces_Iteration1'
                os.chdir(dir)
                new_filename = 'cropped'+str(count)+'.jpg'
                cv2.imwrite(new_filename, cv2.cvtColor(crop_image, cv2.COLOR_RGB2BGR))
                count = count + 1
            else:
                print('Skipped a face..')
    